In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
#input_file = "output_20191008_0336AM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
input_file = "output_20200219_2125PM.csv" #866 files
#input_file = "../data/exogeni/output_20210209_2205PM/output_20210209_2205PM.csv" #OSG Big Topology
df = pd.read_csv(input_file, header = 0)

In [4]:
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']

In [8]:
df_failure = df[ (df.LABEL=='0')]
df_failure.shape

(1411344, 17)

In [266]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.calibration import CalibratedClassifierCV

In [267]:
df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW','FM','RETRIES','FILESIZE','THROUGHPUT','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S','FLOW'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW'])

In [268]:
df_dummy['THROUGHPUT']=df_dummy['THROUGHPUT'].str[0:-4]
df_dummy['THROUGHPUT']=pd.to_numeric(df_dummy['THROUGHPUT'], errors='coerce')
df_dummy = df_dummy.replace(np.nan, 0, regex=True)
y_complete=df_dummy['LABEL']
X_complete=df_dummy.drop(['LABEL'],axis=1)

In [269]:
print(len(df_dummy.columns))
X_complete[0:1]

265


,FM,RETRIES,FILESIZE,THROUGHPUT,SRC_CacheBNL,SRC_CacheCal,SRC_CacheFNAL,SRC_CacheNYU,SRC_CacheSeattle,SRC_CacheSyracuse,...,FLOW_OriginUNL-CacheSeattle,FLOW_OriginUNL-CacheSyracuse,FLOW_OriginUNL-CacheUC,FLOW_OriginUNL-CacheUChic,FLOW_OriginUNL-CacheUNL,FLOW_OriginUNL-OriginCal,FLOW_OriginUNL-OriginFNAL,FLOW_OriginUNL-OriginNCSA,FLOW_OriginUNL-OriginSDSC,FLOW_OriginUNL-OriginSeattle
12659,1,0,1386,203.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [270]:
df_dummy_cate = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S','FLOW'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW'])
#df_dummy_cate['THROUGHPUT']=df_dummy_cate['THROUGHPUT'].str[0:-4]
#df_dummy_cate['THROUGHPUT']=pd.to_numeric(df_dummy_cate['THROUGHPUT'], errors='coerce')
#df_dummy_cate = df_dummy_cate.replace(np.nan, 0, regex=True)
y_cate=df_dummy_cate['LABEL']
X_cate=df_dummy_cate.drop(['LABEL'],axis=1)
X_cate[0:1]

,FM,RETRIES,SRC_CacheBNL,SRC_CacheCal,SRC_CacheFNAL,SRC_CacheNYU,SRC_CacheSeattle,SRC_CacheSyracuse,SRC_CacheUC,SRC_CacheUChic,...,FLOW_OriginUNL-CacheSeattle,FLOW_OriginUNL-CacheSyracuse,FLOW_OriginUNL-CacheUC,FLOW_OriginUNL-CacheUChic,FLOW_OriginUNL-CacheUNL,FLOW_OriginUNL-OriginCal,FLOW_OriginUNL-OriginFNAL,FLOW_OriginUNL-OriginNCSA,FLOW_OriginUNL-OriginSDSC,FLOW_OriginUNL-OriginSeattle
12659,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [271]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [272]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct_class={}
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            #print(f'j={j}, label={label_index}')
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
            correct_class[label]=1
        else:
            correct_class[label]=0
    return correct_class, correct, correct/num 

In [273]:
def print_s_c(c):
    s_c=sorted(c.items(), key=lambda x: x[1], reverse=True)
    num_link=0;num_host=0;link_pos=0;host_pos=0
    for i in s_c:
        name=i[0]
        flag=i[1]
        #print(name+":"+str(flag))
        if 'Link' in name:
            num_link+=1
            if flag==1:
                link_pos+=1
        else:
            num_host+=1
            if flag==1:
                host_pos+=1
    return link_pos*1.0/num_link,host_pos*1.0/num_host

In [274]:
training_dataset={"complete":{"X":X_complete,
                 "y":y_complete},
                 "cate":{"X":X_cate,
                 "y":y_cate}}

In [275]:
def train_dt(classifier):
    df=[df_dummy,df_dummy_cate]
    i=0
    for k, d in training_dataset.items():
        X=d["X"]
        y=d['y']
        clf = classifier
        clf_model=clf.fit(X,y)
        pred=clf.predict(X)
        balanced_accu=balanced_accuracy_score(y, pred)
        f1=f1_score(y, pred,average='weighted')
        print(str(k)+":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
        for j in range(1,4):
            c,correct,accu=accuracy(clf_model, df[i], clf.classes_,j)
            print("Top-"+str(j)+" Accu="+str(accu)) 
        i=i+1

In [276]:
train_dt(DecisionTreeClassifier(max_depth = 20))

complete:balanced_accu=0.18621458394432105:F1-Score=0.3314606159203486
Top-1 Accu=0.6551724137931034
Top-2 Accu=0.7241379310344828
Top-3 Accu=0.7241379310344828
cate:balanced_accu=0.1376416981517295:F1-Score=0.24563996338032348
Top-1 Accu=0.28735632183908044
Top-2 Accu=0.3218390804597701
Top-3 Accu=0.367816091954023


In [277]:
train_dt(RandomForestClassifier(max_depth=20, random_state=0))

complete:balanced_accu=0.8938408289347674:F1-Score=0.8954295440881863
Top-1 Accu=0.9885057471264368
Top-2 Accu=1.0
Top-3 Accu=1.0
cate:balanced_accu=0.225316688667463:F1-Score=0.32562771225557235
Top-1 Accu=0.7011494252873564
Top-2 Accu=0.8045977011494253
Top-3 Accu=0.8275862068965517


In [246]:
%%time
#clf_complete = DecisionTreeClassifier(max_depth=20, random_state=0)
clf_complete = RandomForestClassifier(max_depth=20, random_state=0)
clf_model_complete=clf_complete.fit(X_complete,y_complete)

CPU times: user 1.58 s, sys: 170 ms, total: 1.75 s
Wall time: 1.75 s


In [247]:
score=clf_complete.score(X_complete,y_complete)
print(score)

0.8297673330994972


In [248]:
c,correct,accu=accuracy(clf_model_complete, df_dummy, clf_complete.classes_,1)
print(str(correct)+":"+str(accu))
print(print_s_c(c))

71:0.9102564102564102
(0.9027777777777778, 1.0)


In [249]:
%%time
#clf_cate = DecisionTreeClassifier(max_depth = 20)
clf_cate = RandomForestClassifier(max_depth=20, random_state=0)
clf_model_cate=clf_cate.fit(X_cate,y_cate)

CPU times: user 581 ms, sys: 11.8 ms, total: 593 ms
Wall time: 592 ms


In [250]:
score=clf_cate.score(X_cate,y_cate)
print(score)

0.24354027826493627


In [251]:
c,correct,accu=accuracy(clf_model_cate, df_dummy_cate, clf_cate.classes_,3)
print(str(correct)+":"+str(accu))
print(print_s_c(c))

58:0.7435897435897436
(0.8055555555555556, 0.0)


In [252]:
from imblearn.over_sampling import RandomOverSampler,SMOTE, ADASYN,SMOTENC
from imblearn.combine import SMOTEENN, SMOTETomek

In [253]:
over_sampling = [RandomOverSampler(random_state=0),
        #SMOTE(random_state=0),
        #SMOTEENN(random_state=0),
        #SMOTETomek(random_state=0),
        #SMOTENC(categorical_features=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43], random_state=0)

]

In [254]:
df=[df_dummy,df_dummy_cate]
i=0
for k, d in training_dataset.items():
    X=d["X"]
    y=d['y']
    for s in range(0,len(over_sampling)):
        print("sampling:"+str(s))
        os = over_sampling[s]
        X_resampled, y_resampled = os.fit_resample(X, y)
        clf = RandomForestClassifier(max_depth=20, random_state=0)
        clf_model=clf.fit(X_resampled, y_resampled)
        pred=clf.predict(X)
        balanced_accu=balanced_accuracy_score(y, pred)
        f1=f1_score(y, pred,average='weighted')
        print(str(k)+":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
        for j in range(1,4):
            c,correct,accu=accuracy(clf_model, df[i], clf.classes_,j)
            print("Top-"+str(j)+" Accu="+str(accu)) 
    i=i+1

sampling:0
complete:balanced_accu=0.7290342123846748:F1-Score=0.7394184529586084
Top-1 Accu=1.0
Top-2 Accu=1.0
Top-3 Accu=1.0
sampling:0
cate:balanced_accu=0.2198225926195979:F1-Score=0.1483947095962452
Top-1 Accu=0.8333333333333334
Top-2 Accu=0.9615384615384616
Top-3 Accu=1.0


In [255]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_complete, y_complete)

In [256]:
clf_ros_model=clf.fit(X_resampled,y_resampled)

In [257]:
pred=clf.predict(X_complete)
balanced_accuracy_score(y_complete, pred)

0.7290342123846748

In [258]:
print_s_c(Counter(y_resampled))
#c,correct,accu=accuracy(clf_ros_model, df_dummy, clf.classes_,1)
#c,accu

(0.0, 0.0)

In [259]:
X_resampled, y_resampled = SMOTE().fit_resample(X, y)
clf_smote_model=clf.fit(X_resampled,y_resampled)
pred=clf_smote_model.predict(X)
balanced_accuracy_score(y, pred)

0.21902823317640246

In [260]:
print(sorted(Counter(y_resampled).items()))
c,correct,accu=accuracy(clf_smote_model, df_dummy, clf.classes_,1)
c,accu

[('AlbanyRouter_Link3', 1094), ('AlbanyRouter_Link84', 1094), ('AlbanyRouter_Link85', 1094), ('BostonRouter_Link83', 1094), ('BostonRouter_Link84', 1094), ('CacheBNL_Link6', 1094), ('CacheCal_Link23', 1094), ('CacheFNAL_Link16', 1094), ('CacheNYU_Link4', 1094), ('CacheSDSC_Link21', 1094), ('CacheSeattle_Link24', 1094), ('CacheSyracuse_Link3', 1094), ('CacheUChic_Link14', 1094), ('CacheUNL_Link18', 1094), ('ChicEQCHRouter_Link12', 1094), ('ChicEQCHRouter_Link61', 1094), ('ChicEQCHRouter_Link63', 1094), ('Cleveland_Link85', 1094), ('Cleveland_Link86', 1094), ('DenverRouter_Link1', 1094), ('DenverRouter_Link2', 1094), ('DenverRouter_Link7', 1094), ('ELPasoRouter_Link13', 1094), ('ELPasoRouter_Link2', 1094), ('FNALRouter_Link16', 1094), ('FNALRouter_Link17', 1094), ('FNALRouter_Link62', 1094), ('FNALRouter_Link63', 1094), ('HartfordRouter_Link6', 1094), ('HartfordRouter_Link82', 1094), ('HartfordRouter_Link83', 1094), ('KansasCityRouter_Link1', 1094), ('KansasCityRouter_Link12', 1094), ('K

ValueError: X has 88 features, but DecisionTreeClassifier is expecting 86 features as input.

In [ ]:
smote_nc = SMOTENC(categorical_features=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(x, y)
clf_smotenc_model=clf.fit(X_resampled,y_resampled)
pred=clf_smotenc_model.predict(x)
balanced_accuracy_score(y, pred)

In [ ]:
X_resampled, y_resampled = ADASYN().fit_resample(x, y)
clf_adasyn_model=clf.fit(X_resampled,y_resampled)
pred=clf_adasyn_model.predict(x)
balanced_accuracy_score(y, pred)

In [ ]:
print(sorted(Counter(y_resampled).items()))
c,correct,accu=accuracy(clf_smote_model, df_dummy, clf.classes_,3)
c,accu

In [ ]:
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler
#cc = ClusterCentroids(random_state=0)
cc = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = cc.fit_resample(x, y)
print(sorted(Counter(y_resampled).items()))

In [ ]:
clf_cc_model=clf.fit(X_resampled,y_resampled)
pred=clf_cc_model.predict(x)
balanced_accuracy_score(y, pred)

In [ ]:
c,correct,accu=accuracy(clf_smote_model, df_dummy, clf.classes_,1)
c,accu

In [ ]:
tree.plot_tree(clf.fit(x,y))
dot_data = tree.export_graphviz(clf_model, out_file=None,
                         feature_names=x.columns, 
                         class_names=clf.classes_,
                         filled=True, rounded=True,
                         special_characters=True)
graph = graphviz.Source(dot_data) 
graph.render("simple")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [ ]:
clf_model.classes_

In [ ]:
print(x[:1])

In [ ]:
df_failure.head

In [ ]:
test_prob = clf_model.predict_proba(x[:1])
test_prob

In [ ]:
cross_val_score(clf, x, y, scoring='accuracy',cv=5)

In [ ]:
clf_model.score(x,y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [ ]:
y_pred = clf.fit(x_train, y_train).predict(x_test)

In [ ]:
y_test.shape,y_pred.shape

In [ ]:
balanced_accuracy_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred,average='weighted')

In [ ]:
test_prob = clf_model.predict_proba(x_train[:1])

In [ ]:
test_prob

In [ ]:
clf_model.classes_[26]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=100, max_depth=30, min_samples_split=2, random_state=0)

In [ ]:
clf_rf_model=clf_rf.fit(x, y)

In [ ]:
c,r=accuracy(clf_rf_model, df_dummy, clf.classes_,3)
c,r

In [ ]:
clf_rf_model.score(x,y)

In [ ]:
clf_extra = ExtraTreesClassifier(n_estimators=10, max_depth=20, min_samples_split=2, random_state=0)

In [ ]:
cross_val_score(clf_extra, x, y, cv=5)

In [ ]:
clf_extra_model=clf_extra.fit(x, y)

In [ ]:
c,r=accuracy(clf_extra_model, df_dummy, clf.classes_,3)
c,r

In [ ]:
clf_extra_model.score(x,y)

In [ ]:
y_pred_extra = clf_extra.fit(x_train, y_train).predict(x_test)
balanced_accuracy_score(y_test, y_pred_extra)

In [ ]:
y_pred = clf_model.predict(x)
balanced_accuracy_score(y, y_pred)

In [ ]:
y_pred_rf = clf_rf_model.predict(x)
balanced_accuracy_score(y, y_pred_rf)

In [ ]:
y_pred_extra = clf_extra_model.predict(x)
balanced_accuracy_score(y, y_pred_extra)

In [ ]:
f1_score(y, y_pred_rf,average='weighted')

In [ ]:
df_dummy_1=df_dummy[df_dummy.LABEL=='I2_Link0']

In [ ]:
x_test_1=df_dummy_1.drop(['LABEL'],axis=1)
test_prob_1 = clf_model.predict_proba(x_test_1)
test_prob_1.shape, x_test_1.shape

In [ ]:
test_prob_1.sum(axis=0).argmax()

In [ ]:
clf_model.classes_[26]

In [ ]:
test_prob_1[0]

In [ ]:
test_rf_prob_1 = clf_rf.predict_proba(x_test_1)
test_rf_prob_1.shape, x_test_1.shape

In [ ]:
test_rf_prob_1[100]

In [ ]:
#label_array=np.nanmean(np.where(test_prob_1!=0,test_prob_1,np.nan),axis=0)
#label_array=test_rf_prob_1.mean(axis=0)
label_array=np.true_divide(test_prob_1.sum(0),(test_prob_1!=0).sum(0))

In [ ]:
label_array.argmax()

In [ ]:
clf_model.classes_[44]

In [ ]:
test_rf_prob_1.sum(axis=0).argmax()

In [ ]:
def accuracy_ori(model, test_data, classes):
    num = classes.size
    correct=0
    for label in classes:
        test_data_1=test_data[test_data.LABEL==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        label_pred_1 = classes[label_index]
        label_pred_2 = classes[label_index_sort[-2]] 
        label_pred_3 = classes[label_index_sort[-3]] 
        print(label_array)
        print(label_index,label_index_sort[-1], label_index_sort[-2],label,label_pred_1,label_pred_2)
        if( (label==label_pred_1) | (label==label_pred_2) | (label==label_pred_3)):
            correct=correct+1
    return correct, correct/num

In [ ]:
c,r=accuracy_ori(clf, df_dummy, clf_model.classes_)
c,r

In [ ]:
c,r=accuracy_ori(clf_rf, df_dummy, clf_model.classes_)
c,r

In [ ]:
c,r=accuracy_ori(clf_extra, df_dummy, clf_model.classes_)
c,r

In [ ]:
test_file = "output_20191116_1256PM.csv"
test_df = pd.read_csv(test_file, header = 0)
test_df['FLOW'] = test_df['SRCNODE']+'-'+test_df['DESTNODE']
test_df['FM']=test_df['FAILURE']+test_df['MISSING']
test_df_failure = test_df[ (test_df.FM==1)]
test_dummy = pd.get_dummies(test_df_failure[['SRCNODE', 'DESTNODE','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST'], columns=['SRCNODE', 'DESTNODE'])

In [ ]:
test_dummy.shape

In [ ]:
test_dummy.head

In [ ]:
c,r=accuracy_ori(clf, test_dummy, clf_model.classes_)
c,r

In [ ]:
test_file = "output_20191008_0336AM.csv" #complete
#test_file = "big.csv" #partial
test_df = pd.read_csv(test_file, header = 0)
test_df['FLOW'] = test_df['SRCNODE']+'-'+test_df['DESTNODE']
test_df['FM']=test_df['FAILURE']+test_df['MISSING']
test_df_failure = test_df[ (test_df.FM==1)]

test_df_dummy = pd.get_dummies(test_df_failure[['SRCNODE', 'DESTNODE','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST'], columns=['SRCNODE', 'DESTNODE'])
test_y=test_df_failure['LABEL']
test_x=test_df_dummy.drop(['LABEL'],axis=1)

In [ ]:
%%time
c,r=accuracy(clf, test_df_dummy, clf.classes_,3)
c,r